In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Lambda

import cv2

import numpy as np

import matplotlib.pyplot as plt

import json
import os
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.device(device)

device(type='cuda')

In [3]:
class EnvMapNetDataset(Dataset):
    def __init__(self, images_path, type, minibatch_size, *, force_len=None):
        self.__images_path = os.path.join(images_path, type)
        self.__images = os.listdir(self.__images_path)

        self.__force_len = None
        if force_len:
            self.__force_len = force_len
        else:
            self.__force_len = len(self.__images)

        if self.__force_len % minibatch_size != 0:
            self.__force_len += minibatch_size - (self.__force_len % minibatch_size)

    def __len__(self):
        return self.__force_len

    def __getitem__(self, index):
        image_path = os.path.join(self.__images_path, self.__images[index % len(self.__images)])
        data = np.load(image_path, allow_pickle=True)
        data = torch.from_numpy(np.array([data[:,:,0], data[:,:,1], data[:,:,2]]))
        return data

In [4]:
class L1Loss(nn.Module):
    def __init__(self):
        super(L1Loss, self).__init__()

    def forward(self, orig, pred, mask):
        return (orig * mask - pred * mask).abs().sum()

In [5]:
class L2Loss(nn.Module):
    def __init__(self):
        super(L2Loss, self).__init__()

    def forward(self, orig, pred):
        return (orig - pred).norm().sum()

In [6]:
class FakeRealLoss(nn.Module):
    def __init__(self):
        super(FakeRealLoss, self).__init__()

    def forward(self, orig, pred):
        return -((pred + 1e-9).log() + (1 - orig + 1e-9).log()).sum()

In [7]:
class AdversarialLoss(nn.Module):
    def __init__(self):
        super(AdversarialLoss, self).__init__()

    def forward(self, pred):
        return -(pred + 1e-9).log().sum()

In [8]:
class ORBPatcher:
    def __init__(self, *, key_points=1000):
        self.__orb = cv2.ORB_create(key_points)

    def __call__(self, image):
        key_points, _ = self.__orb.detectAndCompute(image, None)

        res = np.zeros((image.shape[0], image.shape[1]), dtype=np.int)
        for point in key_points:
            x, y = map(np.int, point.pt)
            res[y, x] = 1

        res == 1
        return res.flatten()

In [9]:
class KMeansFiles:
    def __init__(self, orb, *, clusters=5, n_iter=300, partitions=1):
        self.__orb = orb
        self.__clusters = clusters
        self.__n_iter = n_iter
        self.__partitions = partitions

        self.__centers = None

        self.__centers_save_path = os.path.join('KMeans','k_means.bin')

    def fit(self, files):
        partition = (len(files) + self.__partitions - 1) // self.__partitions
        for num in range(self.__partitions):
            print(f'preparing partition {num + 1}')
            tmp = self.__prepare_partition(files[num::self.__partitions])
            torch.save(tmp, os.path.join(f'{num}.pt'))
            print(f'partition {num + 1} prepared\n')
        print('partitions prepared\n')

        if self.__partitions > 1:
            for iteration in range(self.__n_iter):
                print(f'fitting iteration {iteration + 1}')
                for num in range(self.__partitions):
                    tmp = torch.load(os.path.join(f'{num}.pt')).to(device)
                    print(f'partition {num + 1} loaded')
                    self.__fit_partition(tmp)
                    del tmp
                    torch.cuda.empty_cache()
                    print(f'partition {num + 1} fitted')
                print(f'iteration {iteration + 1} fitted\n')
                if (iteration + 1) % 10 == 0:
                    self.save()
                    print('model saved\n')
                print()
        else:
            tmp = torch.load(os.path.join('0.pt')).to(device)
            print(f'partition 0 loaded')
            for iteration in range(self.__n_iter):
                print(f'fitting iteration {iteration + 1}')
                self.__fit_partition(tmp)
                print(f'iteration {iteration + 1} fitted\n')
                torch.cuda.empty_cache()
                if (iteration + 1) % 10 == 0:
                    self.save()
                    print('model saved\n')
                print()

        self.save()
        print('k-means ready')

        for num in range(self.__partitions):
            os.remove(os.path.join('{}.pt'.format(num)))

        self.__centers = self.__centers.to(device)

    def __prepare_partition(self, files):
        return torch.tensor([
            self.__orb((((np.load(file)) + 1) / 2 * 255).astype(np.uint8))
            for file in files
        ])

    def __fit_partition(self, orig):
        x = orig * 1.0

        n, d = x.shape
        if self.__centers is None:
            self.__centers = x[:self.__clusters, :].clone() * 1.0

        x_i = x.view(n, 1, d)
        c_j = self.__centers.view(1, self.__clusters, d)

        D_ij = ((x_i - c_j) ** 2).sum(-1)
        clusters = D_ij.argmin(dim=1).long().view(-1)

        self.__centers.zero_()
        self.__centers.scatter_add_(0, clusters[:, None].repeat(1, d), x)

        Ncl = torch.bincount(clusters, minlength=self.__clusters)
        Ncl = Ncl.type_as(self.__centers)
        Ncl = Ncl.view(self.__clusters, 1)
        self.__centers /= Ncl

    def predict(self, x):
        if self.__centers == None:
            print('k-means wasn\'t fitted')
            return

        n, d = x.shape
        x_i = x.view(n, 1, d)
        c_j = self.__centers.view(1, self.__clusters, d)
        D_ij = ((x_i - c_j) ** 2).sum(-1)
        clusters = D_ij.argmin(dim=1).long().view(-1)

        res = torch.zeros((n, self.__clusters), requires_grad=False).to(device)
        for num, item in enumerate(clusters):
            res[num, item] = 1
        return res

    def save(self):
        torch.save(self.__centers, self.__centers_save_path)

    def load(self):
        self.__centers = torch.load(self.__centers_save_path).to(device)

In [10]:
class ClusterLoss(nn.Module):
    def __init__(self, orb, minibatch_size, *, train_k_means=False, partitions=1, clusters=5):
        super(ClusterLoss, self).__init__()
        self.__orb = orb
        self.__minibatch_size = minibatch_size
        self.__clusters = clusters

        self.__k_means = KMeansFiles(orb, clusters=clusters, partitions=partitions)
        if train_k_means:
            self.__k_means.fit(self.__prepare_data())
            self.__k_means.save()
        else:
            self.__k_means.load()

    def forward(self, images, pred):
        if device != 'cpu':
            images = images.detach().cpu().numpy()
        else:
            images = images.detach().numpy()

        images = np.array([
            np.concatenate([
                image[0].reshape(*image[0].shape, 1),
                image[1].reshape(*image[1].shape, 1),
                image[2].reshape(*image[2].shape, 1),
            ], axis=2) for image in images
        ])
        
        base = self.__k_means.predict(torch.tensor([
            self.__orb(((image + 1) * 255 / 2).astype(np.uint8))
            for image in images
        ]).to(device))

        return -(base * torch.log(nn.Softmax(dim=1)(pred.view(self.__minibatch_size, self.__clusters)) + 1e-9)).sum()

    def __prepare_data(self):
        catalogs = [
            os.path.join('LavalIndoorHDRDatasetReadySmall', 'train'),
            os.path.join('PanoIndoorLDRDatasetReadySmall', 'test'),
            os.path.join('LavalIndoorHDRDatasetReadySmall', 'train'),
            os.path.join('PanoIndoorLDRDatasetReadySmall', 'test')
        ]
        files = []
        for catalog in catalogs:
            files.extend([os.path.join(catalog, file) for file in os.listdir(catalog)])
        return files

In [11]:
class ProjectionLoss(nn.Module):
    def __init__(self, minibatch_size, *, generate_masks=False, masks_count=50, base_shape=(128, 256)):
        super(ProjectionLoss, self).__init__()
        self.__minibatch_size = minibatch_size
        self.__masks_path = 'Masks'

        if generate_masks:
            self.__masks = [
                self.__build_mask(base_shape)
                for _ in range(masks_count)
            ]
            for num, mask in enumerate(self.__masks):
                path = os.path.join(self.__masks_path, str(num + 1))
                np.save(path, mask)
            self.__masks = torch.from_numpy(np.array(self.__masks)).to(device)
        else:
            self.__masks = []
            for i in range(masks_count):
                path = os.path.join(self.__masks_path, str(i + 1) + '.npy')
                self.__masks.append(np.load(path))
            self.__masks = torch.from_numpy(np.array(self.__masks)).to(device)

    def forward(self, orig, pred):
        res = None

        for mask in self.__masks:
            if not res:
                res = torch.abs((orig * mask).sum() - (pred * mask).sum())
            else:
                res += torch.abs((orig * mask).sum() - (pred * mask).sum())

        return res

    def __build_mask(self, base_shape):
        base_shape = (*base_shape, 3)
        image = np.zeros(base_shape)

        for _ in range(int(random.gauss(4, 0.7))):
            scale = random.randint(10, 40)

            h, w, _ = base_shape
            h = h * scale // 100
            w = w * scale // 100

            tmp = np.ones((h, w, 3)) * 255

            angle = random.randint(0, 180)

            M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
            tmp = cv2.warpAffine(tmp, M, (w, h))

            h_c = random.randint(base_shape[0] // 3, 2 * base_shape[0] // 3)
            w_c = random.randint(base_shape[1] // 3, 2 * base_shape[1] // 3)

            top_pad = base_shape[0] - h_c - h // 2
            bottom_pad = h_c - h // 2 - h % 2

            right_pad = base_shape[1] - w_c - w // 2
            left_pad = w_c - w // 2 - w % 2

            tmp = np.pad(tmp, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)))

            image = cv2.add(image, tmp)

        res = (image == 0).astype(np.int)
        res = np.array([res[:,:,0], res[:,:,1], res[:,:,2]])

        return [res.copy() for _ in range(self.__minibatch_size)]

In [12]:
class EnvMapNetLoss(nn.Module):
    def __init__(self, orb, minibatch_size, *, train_k_means=False, generate_masks=False):
        super(EnvMapNetLoss, self).__init__()
        self.L1Loss = L1Loss()
        self.L2Loss = L2Loss()
        self.AdversarialLoss = AdversarialLoss()
        self.ClusterLoss = ClusterLoss(orb, minibatch_size, train_k_means=train_k_means)
        self.ProjectionLoss = ProjectionLoss(minibatch_size, generate_masks=generate_masks)

    def forward(self, orig, mask, pred, discriminator_pred1, discriminator_pred2):
        return\
            0.5 * self.L1Loss(orig, pred, mask) +\
            0.01 * self.L2Loss(orig, pred) +\
            self.ProjectionLoss(orig, pred) +\
            self.AdversarialLoss(discriminator_pred1) +\
            self.ClusterLoss(orig, discriminator_pred2)

In [13]:
class DiscriminatorLoss(nn.Module):
    def __init__(self, orb, minibatch_size, *, train_k_means=False):
        super(DiscriminatorLoss, self).__init__()
        self.FakeRealLoss = FakeRealLoss()
        self.ClusterLoss = ClusterLoss(orb, minibatch_size, train_k_means=train_k_means)

    def forward(self, orig, pred_1, pred_2, pred_g):
        return\
            self.FakeRealLoss(pred_1, pred_g) +\
            self.ClusterLoss(orig, pred_2)

In [14]:
class EnvMapNetConvBlock(nn.Module):
    def __init__(self, channels, *, sets_count=5, conv_channels=16):
        super(EnvMapNetConvBlock, self).__init__()
        self.__blocks = nn.Sequential(*[
            item for sublist in [
                self.__get_set(channels, conv_channels, i)
                for i in range(sets_count)
            ] for item in sublist
        ])

    def forward(self, x):
        short_cut = x
        counter = 0
        for block in self.__blocks:
            x = block(x)
            counter += 1
            if counter == 3:
                x = torch.cat((x, short_cut), 1)
                short_cut = x
                counter = 0
        return x
    
    def __get_set(self, in_channels, conv_channels, i):
        channels = in_channels + conv_channels * i
        return [
            nn.BatchNorm2d(channels),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv2d(
                channels,
                conv_channels,
                kernel_size=(3, 3),
                padding=(1, 1),
                padding_mode='circular',
            )
        ]

In [15]:
class EnvMapNetDownsampleBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EnvMapNetDownsampleBlock, self).__init__()
        self.__conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=(3, 3),
            padding=(1, 1),
            padding_mode='circular',
        )
        self.__downsample = nn.AvgPool2d((2, 2))

    def forward(self, x):
        x = self.__conv(x)
        x = self.__downsample(x)
        return x

In [16]:
class EnvMapNetUpsampleBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EnvMapNetUpsampleBlock, self).__init__()
        self.__conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=(3, 3),
            padding=(1, 1),
            padding_mode='circular',
        )
        self.__upsample = nn.Upsample(scale_factor=2)

    def forward(self, x):
        x = self.__upsample(x)
        x = self.__conv(x)
        return x

In [17]:
class Generator(nn.Module):
    def __init__(self,
                 dk_orig, uk_orig, *, sets_count=5,
                 conv_channels=16, neck_channels=64):
        super(Generator, self).__init__()
        
        dk = [3] + dk_orig
        self.__downsampling_blocks = nn.Sequential(*[
            item for sublist in [
                [
                    EnvMapNetConvBlock(
                        dk[i - 1],
                        sets_count=sets_count,
                        conv_channels=conv_channels
                    ),
                    EnvMapNetDownsampleBlock(
                        dk[i - 1] + sets_count * conv_channels,
                        dk[i]
                    )
                ]
                for i in range(1, len(dk))
            ] for item in sublist
        ])

        self.__neck = nn.Conv2d(
            dk[-1],
            neck_channels,
            kernel_size=(1, 1),
        )

        uk = [neck_channels] + uk_orig
        self.__upsampling_blocks = nn.Sequential(*[
            item for sublist in [
                [
                    EnvMapNetUpsampleBlock(
                        uk[i - 1] + (i != 1) *
                            (sets_count * conv_channels),
                        uk[i]),
                    EnvMapNetConvBlock(
                        uk[i],
                        sets_count=sets_count,
                        conv_channels=conv_channels
                    )
                ]
                for i in range(1, len(uk))
            ] for item in sublist
        ])

        self.__out = nn.Conv2d(
            uk[-1] + sets_count * conv_channels,
            3,
            kernel_size=(3, 3),
            padding=(1, 1),
            padding_mode='circular',
        )

    def forward(self, x):
        for block in self.__downsampling_blocks:
            x = block(x)

        x = self.__neck(x)

        for block in self.__upsampling_blocks:
            x = block(x)

        x = self.__out(x)

        return torch.tanh(x)

In [18]:
class DiscriminatorResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, *, sets_count=2):
        super(DiscriminatorResidualBlock, self).__init__()
        self.__avg = nn.AvgPool2d((2, 2))
        self.__conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=(3, 3),
            padding=(1, 1),
            padding_mode='circular',
        )

        self.__blocks = nn.Sequential(*[
            item for sublist in [
                self.__get_set(in_channels if i == 0 else out_channels, out_channels)
                for i in range(sets_count)
            ] for item in sublist
        ])

    def forward(self, inp):
        sc = self.__avg(inp)
        sc = self.__conv(sc)
        sc = nn.functional.pad(
            sc, (sc.shape[-1] // 2, sc.shape[-1] // 2, sc.shape[-2] // 2, sc.shape[-2] // 2)
        )

        x = inp
        for block in self.__blocks:
            x = block(x)

        return x + sc
    
    def __get_set(self, in_channels, out_channels):
        return [
            nn.BatchNorm2d(in_channels),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=(3, 3),
                padding=(1, 1),
                padding_mode='circular',
            )
        ]

In [19]:
class Discriminator(nn.Module):
    def __init__(self, ak_orig, *, sets_count=2, base_shape=(128, 256), clusters=5):
        super(Discriminator, self).__init__()

        ak = [3] + ak_orig

        self.__blocks = nn.Sequential(*[
            DiscriminatorResidualBlock(ak[i - 1], ak[i], sets_count=sets_count)
            for i in range(1, len(ak))
        ])
        
        self.__out1_conv = nn.Conv2d(
            ak[-1],
            1,
            kernel_size=base_shape,
        )
        self.__out2_conv = nn.Conv2d(
            ak[-1],
            clusters,
            kernel_size=base_shape,
        )

    def forward(self, x):
        for block in self.__blocks:
            x = block(x)
        return nn.Sigmoid()(self.__out1_conv(x)), self.__out2_conv(x)

In [20]:
dk = [64, 128, 128, 128, 256, 256, 512]
uk = [512, 256, 256, 128, 128, 128, 64]
ak = [64, 128, 256, 256, 256, 256, 256]

In [21]:
minibatch_size = 4
epochs = 500

In [22]:
generator = nn.DataParallel(Generator(dk, uk)).to(device)
if os.path.exists(os.path.join('generator.bin')):
    model.load_state_dict(torch.load(os.path.join('generator.bin')))
discriminator = nn.DataParallel(Discriminator(ak)).to(device)
if os.path.exists(os.path.join('discriminator.bin')):
    model.load_state_dict(torch.load(os.path.join('discriminator.bin')))

In [23]:
generator_train_dataloader = DataLoader(
    EnvMapNetDataset(os.path.join('LavalIndoorHDRDatasetReadySmall'), 'train', minibatch_size), batch_size=minibatch_size, shuffle=True
)
generator_test_dataloader = DataLoader(
    EnvMapNetDataset(os.path.join('LavalIndoorHDRDatasetReadySmall'), 'test', minibatch_size), batch_size=minibatch_size, shuffle=True
)
discriminator_train_dataloader = DataLoader(
    EnvMapNetDataset(os.path.join('PanoIndoorLDRDatasetReadySmall'), 'train', minibatch_size, force_len=len(generator_train_dataloader.dataset)), batch_size=minibatch_size, shuffle=True
)
discriminator_test_dataloader = DataLoader(
    EnvMapNetDataset(os.path.join('PanoIndoorLDRDatasetReadySmall'), 'test', minibatch_size, force_len=len(generator_test_dataloader.dataset)), batch_size=minibatch_size, shuffle=True
)

In [24]:
orb = ORBPatcher()
generator_loss = EnvMapNetLoss(orb, minibatch_size)
discriminator_loss = DiscriminatorLoss(orb, minibatch_size)

In [25]:
generator_optimiser = torch.optim.Adam(generator.parameters(), lr=0.0002)
discriminator_optimiser = torch.optim.Adam(discriminator.parameters(), lr=0.0002)

In [26]:
mask = torch.zeros((minibatch_size, 3, 128, 256)).to(device)
to_change = mask[:,:,128 // 3:2 * 128 // 3, 256 // 3:2 * 256 // 3]
mask[:,:,128 // 3:2 * 128 // 3, 256 // 3:2 * 256 // 3] = torch.ones_like(to_change)

In [27]:
def train_loop(generator,
               generator_train_dataloader,
               generator_loss,
               generator_optimiser,
               discriminator,
               discriminator_train_dataloader,
               discriminator_loss,
               discriminator_optimiser
              ):
    generator.train()
    discriminator.train()

    g_size = len(generator_train_dataloader.dataset)
    d_size = len(discriminator_train_dataloader.dataset)

    for batch, (g_data, d_data) in enumerate(zip(
        generator_train_dataloader,
        discriminator_train_dataloader
    )):

        g_data = g_data.to(device)
        rand = torch.zeros_like(g_data).to(device).uniform_(-1, 1) * (1 - mask)
        to_apply = (g_data * mask + rand).float()
        g_pred = generator(to_apply)

        discriminator.eval()
        d_pred1, d_pred2 = discriminator(g_pred)
        discriminator.train()

        g_loss = generator_loss(g_data, mask, g_pred, d_pred1, d_pred2)
        generator_optimiser.zero_grad()
        g_loss.backward()
        generator_optimiser.step()

        if batch > 0 and batch % 10 == 0:
            loss, current = g_loss.item(), batch * len(g_data)
            print(f"loss: {loss:>7f}  [{current:>5d}/{g_size:>5d}]", end='\t')

        del g_data, rand, to_apply, g_pred, d_pred1, d_pred2, g_loss
        torch.cuda.empty_cache()

        d_data = d_data.float().to(device)
        
        generator.eval()
        rand = torch.zeros_like(d_data).to(device).uniform_(-1, 1) * (1 - mask)
        to_apply = (d_data * mask + rand).float()
        g_pred = generator(to_apply)
        generator.train()

        d_pred1, d_pred2 = discriminator(d_data)
        g_pred1, g_pred2 = discriminator(g_pred)

        d_loss = discriminator_loss(d_data, d_pred1, d_pred2, g_pred1)

        discriminator_optimiser.zero_grad()
        d_loss.backward()
        discriminator_optimiser.step()

        if batch > 0 and batch % 10 == 0:
            loss, current = d_loss.item(), batch * len(d_data)
            print(f"loss: {loss:>7f}  [{current:>5d}/{d_size:>5d}]")

        del d_data, d_pred1, d_pred2, d_loss, g_pred, g_pred1, g_pred2, rand, to_apply
        torch.cuda.empty_cache()


def test_loop(generator,
              generator_test_dataloader,
              generator_loss,
              discriminator,
              discriminator_test_dataloader,
              discriminator_loss
             ):
    generator.eval()
    discriminator.eval()

    g_size = len(generator_test_dataloader.dataset)
    d_size = len(discriminator_test_dataloader.dataset)
    g_loss, d_loss = 0, 0

    with torch.no_grad():
        for g_data, d_data in zip(
            generator_test_dataloader,
            discriminator_test_dataloader
        ):

            g_data = g_data.to(device)
            rand = torch.zeros_like(g_data).to(device).uniform_(-1, 1) * (1 - mask)
            to_apply = (g_data * mask + rand).float()
            g_pred = generator(to_apply)
            d_pred1, d_pred2 = discriminator(g_pred)
            g_loss += generator_loss(g_data, mask, g_pred, d_pred1, d_pred2).item()

            del g_data, rand, to_apply, g_pred, d_pred1, d_pred2
            torch.cuda.empty_cache()

            d_data = d_data.to(device).float()
            rand = torch.zeros_like(d_data).to(device).uniform_(-1, 1) * (1 - mask)
            to_apply = (d_data * mask + rand)
            g_pred = generator(to_apply)

            d_pred1, d_pred2 = discriminator(d_data)
            g_pred1, g_pred2 = discriminator(g_pred)

            d_loss += discriminator_loss(d_data, d_pred1, d_pred2, g_pred1).item()

            del d_data, d_pred1, d_pred2, g_pred, g_pred1, g_pred2, rand, to_apply
            torch.cuda.empty_cache()

    g_loss /= g_size
    d_loss /= d_size

    print("Test Error:")
    print(f" Avg generator loss: {g_loss:>8f}")
    print(f" Avg discriminator loss: {d_loss:>8f}")
    print()

In [ ]:
for t in range(epochs):
    print(f"Epoch {t + 1} started")
    train_loop(
        generator,
        generator_train_dataloader,
        generator_loss,
        generator_optimiser,
        discriminator,
        discriminator_train_dataloader,
        discriminator_loss,
        discriminator_optimiser
    )
    print(f"Epoch {t + 1} trained")
    test_loop(
        generator,
        generator_test_dataloader,
        generator_loss,
        discriminator,
        discriminator_test_dataloader,
        discriminator_loss
    )
    print(f"Epoch {t + 1} tested\n")
    if (t + 1) % 10 == 0:
        torch.save(generator.state_dict(), os.path.join('generator.bin'))
        torch.save(discriminator.state_dict(), os.path.join('discriminator.bin'))
        print(f"Model by epoch {t + 1} saved\n")

Epoch 1 started


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information

loss: 1026790.904319  [   40/ 1924]	loss: 82.893066  [   40/ 1924]
loss: 425943.967494  [   80/ 1924]	loss: 82.893066  [   80/ 1924]
loss: 666271.959024  [  120/ 1924]	loss: 124.339600  [  120/ 1924]
loss: 715800.574464  [  160/ 1924]	loss: 82.893066  [  160/ 1924]
loss: 855999.054968  [  200/ 1924]	loss: 82.893066  [  200/ 1924]
loss: 479575.910125  [  240/ 1924]	loss: 82.893066  [  240/ 1924]
loss: 868989.089734  [  280/ 1924]	loss: 82.893066  [  280/ 1924]
loss: 873286.945349  [  320/ 1924]	loss: 82.893066  [  320/ 1924]
loss: 484448.753839  [  360/ 1924]	loss: 82.893066  [  360/ 1924]
loss: 750198.785272  [  400/ 1924]	loss: 82.893066  [  400/ 1924]
loss: 1128436.706616  [  440/ 1924]	loss: 82.893066  [  440/ 1924]
loss: 308621.446960  [  480/ 1924]	loss: 82.893066  [  480/ 1924]
loss: 327102.103103  [  520/ 1924]	loss: 82.893066  [  520/ 1924]
loss: 677506.113343  [  560/ 1924]	loss: 82.893066  [  560/ 1924]
loss: 730887.698041  [  600/ 1924]	loss: 82.893066  [  600/ 1924]
loss: 2

loss: 6431805.435708  [  960/ 1924]	loss: 82.893066  [  960/ 1924]
loss: 7146694.050384  [ 1000/ 1924]	loss: 82.893066  [ 1000/ 1924]
loss: 7769262.161744  [ 1040/ 1924]	loss: 82.893066  [ 1040/ 1924]
loss: 7445417.687695  [ 1080/ 1924]	loss: 82.893066  [ 1080/ 1924]
loss: 6813501.412719  [ 1120/ 1924]	loss: 82.893066  [ 1120/ 1924]
loss: 6115698.208238  [ 1160/ 1924]	loss: 82.893066  [ 1160/ 1924]
loss: 7405847.377120  [ 1200/ 1924]	loss: 82.893066  [ 1200/ 1924]
loss: 7012719.568784  [ 1240/ 1924]	loss: 82.893066  [ 1240/ 1924]
loss: 8123885.485108  [ 1280/ 1924]	loss: 82.893066  [ 1280/ 1924]
loss: 6374033.699177  [ 1320/ 1924]	loss: 82.893066  [ 1320/ 1924]
loss: 7564304.353986  [ 1360/ 1924]	loss: 82.893066  [ 1360/ 1924]
loss: 7492116.981513  [ 1400/ 1924]	loss: 82.893066  [ 1400/ 1924]
loss: 6517937.662782  [ 1440/ 1924]	loss: 82.893066  [ 1440/ 1924]
loss: 7004872.402819  [ 1480/ 1924]	loss: 82.893066  [ 1480/ 1924]
loss: 6661295.242325  [ 1520/ 1924]	loss: 82.893066  [ 1520/ 1

loss: 7261848.686744  [  720/ 1924]	loss: 82.893066  [  720/ 1924]
loss: 7871824.579380  [  760/ 1924]	loss: 82.893066  [  760/ 1924]
loss: 7400706.426199  [  800/ 1924]	loss: 82.893066  [  800/ 1924]
loss: 10871810.809192  [  840/ 1924]	loss: 82.893066  [  840/ 1924]
loss: 7408459.734507  [  880/ 1924]	loss: 82.893066  [  880/ 1924]
loss: 8175576.140449  [  920/ 1924]	loss: 82.893066  [  920/ 1924]
loss: 6486268.107373  [  960/ 1924]	loss: 82.893066  [  960/ 1924]
loss: 6974957.576678  [ 1000/ 1924]	loss: 82.893066  [ 1000/ 1924]
loss: 7189967.835729  [ 1040/ 1924]	loss: 82.893066  [ 1040/ 1924]
loss: 7928795.880152  [ 1080/ 1924]	loss: 82.893066  [ 1080/ 1924]
loss: 6858382.631690  [ 1120/ 1924]	loss: 82.893066  [ 1120/ 1924]
loss: 6943081.812048  [ 1160/ 1924]	loss: 82.893066  [ 1160/ 1924]
loss: 9088873.071432  [ 1200/ 1924]	loss: 82.893066  [ 1200/ 1924]
loss: 8840730.348630  [ 1240/ 1924]	loss: 82.893066  [ 1240/ 1924]
loss: 7301213.538495  [ 1280/ 1924]	loss: 82.893066  [ 1280/ 

loss: 9944626.353286  [  480/ 1924]	loss: 82.893066  [  480/ 1924]
loss: 8796356.666637  [  520/ 1924]	loss: 82.893066  [  520/ 1924]
loss: 6814028.456870  [  560/ 1924]	loss: 82.893066  [  560/ 1924]
loss: 6569271.696748  [  600/ 1924]	loss: 82.893066  [  600/ 1924]
loss: 6945157.896894  [  640/ 1924]	loss: 82.893066  [  640/ 1924]
loss: 7811653.350255  [  680/ 1924]	loss: 82.893066  [  680/ 1924]
loss: 7587407.146849  [  720/ 1924]	loss: 82.893066  [  720/ 1924]
loss: 6903084.596602  [  760/ 1924]	loss: 82.893066  [  760/ 1924]
loss: 7793574.246031  [  800/ 1924]	loss: 82.893066  [  800/ 1924]
loss: 7226487.489013  [  840/ 1924]	loss: 82.893066  [  840/ 1924]
loss: 7482107.395684  [  880/ 1924]	loss: 82.893066  [  880/ 1924]
loss: 6994535.246529  [  920/ 1924]	loss: 82.893066  [  920/ 1924]
loss: 8542779.515697  [  960/ 1924]	loss: 82.893066  [  960/ 1924]
loss: 7277838.261872  [ 1000/ 1924]	loss: 82.893066  [ 1000/ 1924]
loss: 7055292.766674  [ 1040/ 1924]	loss: 82.893066  [ 1040/ 1

loss: 7439693.389248  [  240/ 1924]	loss: 82.893066  [  240/ 1924]
loss: 8659325.422474  [  280/ 1924]	loss: 82.893066  [  280/ 1924]
loss: 6507545.648063  [  320/ 1924]	loss: 82.893066  [  320/ 1924]
loss: 5742993.281178  [  360/ 1924]	loss: 82.893066  [  360/ 1924]
loss: 6939220.557255  [  400/ 1924]	loss: 82.893066  [  400/ 1924]
loss: 8614372.405067  [  440/ 1924]	loss: 82.893066  [  440/ 1924]
loss: 8055093.326140  [  480/ 1924]	loss: 82.893066  [  480/ 1924]
loss: 7560854.646934  [  520/ 1924]	loss: 82.893066  [  520/ 1924]
loss: 7344598.231635  [  560/ 1924]	loss: 82.893066  [  560/ 1924]
loss: 7352950.521671  [  600/ 1924]	loss: 82.893066  [  600/ 1924]
loss: 6926397.781005  [  640/ 1924]	loss: 82.893066  [  640/ 1924]
loss: 5928855.227494  [  680/ 1924]	loss: 82.893066  [  680/ 1924]
loss: 6972130.680483  [  720/ 1924]	loss: 82.893066  [  720/ 1924]
loss: 6600233.678211  [  760/ 1924]	loss: 82.893066  [  760/ 1924]
loss: 8762727.912876  [  800/ 1924]	loss: 82.893066  [  800/ 1

loss: 7876555.964792  [   40/ 1924]	loss: 82.893066  [   40/ 1924]
loss: 7403364.159092  [   80/ 1924]	loss: 82.893066  [   80/ 1924]
loss: 6827324.788428  [  120/ 1924]	loss: 82.893066  [  120/ 1924]
loss: 8490884.974644  [  160/ 1924]	loss: 82.893066  [  160/ 1924]
loss: 7842232.715878  [  200/ 1924]	loss: 82.893066  [  200/ 1924]
loss: 7239753.226328  [  240/ 1924]	loss: 82.893066  [  240/ 1924]
loss: 7349774.212884  [  280/ 1924]	loss: 82.893066  [  280/ 1924]
loss: 7330005.198412  [  320/ 1924]	loss: 82.893066  [  320/ 1924]
loss: 6888562.263424  [  360/ 1924]	loss: 82.893066  [  360/ 1924]
loss: 7853108.333452  [  400/ 1924]	loss: 82.893066  [  400/ 1924]
loss: 8767648.378580  [  440/ 1924]	loss: 82.893066  [  440/ 1924]
loss: 7190905.947018  [  480/ 1924]	loss: 82.893066  [  480/ 1924]
loss: 7220346.807355  [  520/ 1924]	loss: 82.893066  [  520/ 1924]
loss: 9231135.786626  [  560/ 1924]	loss: 82.893066  [  560/ 1924]
loss: 7686401.527747  [  600/ 1924]	loss: 82.893066  [  600/ 1

loss: 9858365.071904  [ 1800/ 1924]	loss: 82.893066  [ 1800/ 1924]
loss: 7726374.943456  [ 1840/ 1924]	loss: 82.893066  [ 1840/ 1924]
loss: 5991273.504179  [ 1880/ 1924]	loss: 82.893066  [ 1880/ 1924]
loss: 7003835.159065  [ 1920/ 1924]	loss: 82.893066  [ 1920/ 1924]
Epoch 27 trained
Test Error:
 Avg generator loss: 1845224.124328
 Avg discriminator loss: 62.169800

Epoch 27 tested

Epoch 28 started
loss: 6248729.625138  [   40/ 1924]	loss: 82.893066  [   40/ 1924]
loss: 6713698.985745  [   80/ 1924]	loss: 82.893066  [   80/ 1924]
loss: 9261546.214182  [  120/ 1924]	loss: 82.893066  [  120/ 1924]
loss: 7807348.408366  [  160/ 1924]	loss: 82.893066  [  160/ 1924]
loss: 6718766.225941  [  200/ 1924]	loss: 82.893066  [  200/ 1924]
loss: 7532836.037712  [  240/ 1924]	loss: 82.893066  [  240/ 1924]
loss: 10122218.900058  [  280/ 1924]	loss: 82.893066  [  280/ 1924]
loss: 7756084.850226  [  320/ 1924]	loss: 82.893066  [  320/ 1924]
loss: 5946305.271995  [  360/ 1924]	loss: 82.893066  [  360/